# Introduction

This notebook demonstrates retreiving information from SQL Databases using Azure OpenAI. The notebook is a part of the larger work on End-to-End virtual assisstant demo.

**Background**

We assume the role of a regional sales manager, who is interesed in learning more about Sales of different Surface products, price points etc. Information on sales is available across multiple tables in the SQL Database. In this notebook, we illustrate how to retrieve information available in SQL databases through natural language questioning. Specifically:

1. We ask (sales related) questions in natural language
2. Pass the question to Azure OpenAI as prompt. The completion we get in return is a SQL query to retrieve the answer
3. We run this SQL query on our database to retrieve an answer
4. Finally, we pass the question and (SQL retreived answer) to Azure OpenAI. The completion we get in return is the answer in natural language.

For information on retreiving information from unstructured data or for the integrated demo, please refer to other material within the bundle.

In [1]:
# Install Packages
import openai
import pyodbc
import pandas as pd
import time

C:\Users\mejani\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


First, we set up Azure OpenAI and SQL Server

In [2]:
def run_chat_resouce():
    '''
    Setup Azure OpenAI
    '''
    API_KEY = # SET YOUR OWN API KEY HERE
    RESOURCE_ENDPOINT = 
    API_VERSION = "2023-03-15-preview" 
    
    return API_KEY, RESOURCE_ENDPOINT, API_VERSION
    
def connect_sql_server(database):
    '''
    Setup SQL Server
    '''
    server = 'aoai-db-server.database.windows.net'  
    username = 
    password = 
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    return cnxn


def run_sql_query(completion):
    '''
    Function to run the generated SQL Query on SQL server and retrieve output.
    Input: AOAI completion (SQL Query)
    Output: Pandas dataframe containing results of the query run
    
    '''
    cnxn = connect_sql_server('aoai-db')
    df = pd.read_sql(completion, cnxn)
    return df


openai.api_type = "azure"
openai.api_key = run_chat_resouce()[0]
openai.api_base = run_chat_resouce()[1]
openai.api_version = run_chat_resouce()[2]


# Prompt Engineering

Let us look at the prompts to pass to Azure OpenAI for NL to SQL task. You find this prompt in **generate_nl_to_sql()**.

*System Prompt*

The first step is preparing the system prompt. This sets up the overall tone for GPT to answer Sales question. It also sets informs the bot the kind of questions it is authorized to answer.Following are some characteristics of our system prompt:
1. We first generally describe the bots role (you are a SQL Programmer Assistant)
2. We provide some additional details on the role:

    a. Specify the SQL version to be used
    
    b. Insturctions for what to do if a valid query cannot be produced
    
    c. Instructions on what kind of questions to not answer
    
    d. Any other specific instruction.
    
    
3. Since we are solving an NL to SQL problem, we need to provide the schema of the Sales database next. We provide column names along with datatypes, and relationship information by declaring primary and foreign keys
4. Lastly, we provide some few shot examples. After experimentation on zero shot, few shot examples are added based on the types of queries we see most errors on. We make sure to pass these examples in the "user" and "assistant" role setup. 

*User Prompt*

The next step is to prepare the user prompt. This is just the NL question to be asked. Make sure to follow some basic guidelines while phrasing questions:

1. Be specific in what you are asking
2. Make sure to phrase the question simply. Consider paraphrasing or breaking the question down into smaller segments in the question is involved.


Our next prompt is to convert the query retreived answer into better format. You find this prompt in **generate_sql_to_nl()**

*System Prompt*

Since this is an easier task, the prompt is relatively simpler.

1. Set the overall role for the bot
2. Provide detailed instructions on the task to be performed (convert to natural language. Convert table to html table)
3. provide guidance on what not to do (do not return markdown format)

*User Prompt*

The user prompt for this setup is just passing the original natural language question along with the SQL retreived answer.

In [3]:
def generate_nl_to_sql(prompt_in, history = []):
    '''
    This GPT4 engine is setup for NLtoSQL tasks on the Sales DB.
    Input: NL question related to sales
    Output: SQL query to run on the sales database
    '''

    messages=[
            {"role": "system", "content": """ You are a SQL programmer Assistant.Your role is to generate SQL code (SQL Server) to retrieve an answer to a natural language query. Make sure to disambiguate column names when creating queries that use more than one table. If a valid SQL query cannot be generated, only say "ERROR:" followed by why it cannot be generated.
                  Do not answer any questions on inserting or deleting rows from the table. Instead, say "ERROR: I am not authorized to make changes to the data"

                  Use the following sales database schema to write SQL queries:
                  Customers(cust_id INTEGER, cust_name VARCHAR, cust_email VARCHAR, cust_phone VARCHAR, cust_address VARCHAR, PRIMARY KEY (cust_id))
                  Products(prod_id INTEGER,prod_name varchar, price FLOAT, category VARCHAR, PRIMARY KEY(prod_id))
                  Stock(prod_id INTEGER, merchant_id INTEGER, stock INTEGER, PRIMARY KEY(prod_id, merchant_id), FOREIGN KEY(merchant_id, prod_id))
                  Merchants(merchant_id INTEGER, merchant_name VARCHAR, merchant_region VARCHAR, merchant_address VARCHAR, PRIMARY KEY(merchant_id))
                  Sales(sale_id INTEGER, cust_id INTEGER , merchant_id INTEGER , date TIMESTAMP, total_price FLOAT, PRIMARY KEY(sale_id),FOREIGN KEY(cust_id,merchant_id))
                  Sales_Detail(sales_id INTEGER, prod_id INTEGER, quantity INTEGER, PRIMARY KEY(sales_id,prod_id), FOREIGN KEY(sales_id,prod_id))

                  Examples:
                  User: List all Surface accessories, along with their prices. SQL Code:
                  Assistant: SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';
                  User: Which is the cheapest Surface device? SQL Code:
                  Assistant: SELECT TOP 1 prod_name, price FROM Products WHERE prod_name like '%Surface%' ORDER BY price ASC;
                  User: How many Surface Laptop 5 does GadgetWorld have?
                  Assistant: SELECT Merchants.merchant_id, Merchants.merchant_name, SUM(stock) as total_stock FROM Stock JOIN Merchants ON Stock.merchant_id = Merchants.merchant_id WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%' and merchant_name like '%GadgetWorld%') GROUP BY Merchants.merchant_id, Merchants.merchant_name;
                  User: how many surface devices were sold last week?
                  Assistant: SELECT Sum(sales_detail.quantity) AS total_surface_devices_sold FROM sales_detail JOIN sales ON sales_detail.sales_id = sales.sale_id JOIN products ON sales_detail.prod_id = products.prod_id WHERE products.prod_name LIKE '%Surface%' AND sales.date >= Dateadd(wk, Datediff(wk, 0, Getdate()) - 1, 0) AND sales.date < Dateadd(wk, Datediff(wk, 0, Getdate()), 0); 
            """}
        ]

    messages.extend(history)
    messages.append({"role": "user", "content": prompt_in})
    
    response = openai.ChatCompletion.create(
        engine="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages=messages,
        temperature=0,
        max_tokens=2000,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return response['choices'][0]['message']['content']

def generate_sql_to_nl(prompt_in):
    '''
    This GPT4 engine is setup for SQLtoNL tasks on the Sales DB.
    Input: Original question asked. Answer retreived from running SQL query.
    Output: Natural language sentence(s).
    '''
    
    response = openai.ChatCompletion.create(
        engine="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages=[
            {"role": "system", "content": """You are bot that takes question-answer pairs and converts the answer to natural language. For tabular information return it as an html table. Do not return markdown format."""},
            {"role": "user", "content": prompt_in},
        ],
        temperature=0,
        max_tokens=2000,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return response['choices'][0]['message']['content']

# Examples

In the subsequent cells, we demonstrate the setup in action. We first try out a simple question. Next, we try out a more involved question, which looks at information from multiple tables. Lastly, we see a multi-turn question i.e a question which requires context from the question previously asked.

**Handling Multi-turn questions in prompts**

In some cases, the user breaks down the question in two multiple parts as the conversation continues. In this case, GPT would require context from the history to be able to comprehend and correctly answer the current question. As a result, we add this extra information to the prompt before sending our current question. Specifically, 
1. We first append our conversation history to the prompt. We keep the "user", "assistant" format we have been following so far with our prompt. Within "user", we add the qustion asked in "user", and the NL answer recieved in "assistant". Make sure to keep the order of Q&A as it occured in the conversation.
2. Finally, we add our current question to the prompt, and pass this newly updated prompt to generate_nl_to_sql()

### Example 1: Products - A simple NLtoSQL query

Our user input is the NL question, as an output, Azure OpenAI generates as completion the SQL query we should run on the Sales database to answer this question

In [4]:
question_products = 'List our all products with their prices.'
completion_products = generate_nl_to_sql(question_products)
print(completion_products)

SELECT prod_name, price FROM Products;


We run the SQL query on our database. We retrive the information as a pandas dataframe

In [5]:
sql_output_products = run_sql_query(completion_products)
print(sql_output_products)

             prod_name    price
0        Surface Pro 9   999.99
1     Surface Laptop 5   899.99
2   Surface Slim Pen 2    94.99
3    Surface Laptop SE   299.99
4  Surface Laptop Go 2   599.99
5         Surface Go 3   384.99
6    Surface Studio 2+  1199.99
7          Surface Pen    99.99
8        Surface Duo 2  1099.99
9         Surface Dock   199.99


We ask OpenAI to convert this into a more "Chat-setting friendly" answer. We pass the question, and the answer to Azure OpenAI and ask it to convert the informtaion to an HTML table. In case of a non table asnwer, We ask Azure OpenAI to convert the answer to natural language format.

In [6]:
nl_output = generate_sql_to_nl(str(question_products +  str(sql_output_products.to_dict('list')) ))
print(nl_output)

<table>
  <tr>
    <th>Product Name</th>
    <th>Price</th>
  </tr>
  <tr>
    <td>Surface Pro 9</td>
    <td>$999.99</td>
  </tr>
  <tr>
    <td>Surface Laptop 5</td>
    <td>$899.99</td>
  </tr>
  <tr>
    <td>Surface Slim Pen 2</td>
    <td>$94.99</td>
  </tr>
  <tr>
    <td>Surface Laptop SE</td>
    <td>$299.99</td>
  </tr>
  <tr>
    <td>Surface Laptop Go 2</td>
    <td>$599.99</td>
  </tr>
  <tr>
    <td>Surface Go 3</td>
    <td>$384.99</td>
  </tr>
  <tr>
    <td>Surface Studio 2+</td>
    <td>$1199.99</td>
  </tr>
  <tr>
    <td>Surface Pen</td>
    <td>$99.99</td>
  </tr>
  <tr>
    <td>Surface Duo 2</td>
    <td>$1099.99</td>
  </tr>
  <tr>
    <td>Surface Dock</td>
    <td>$199.99</td>
  </tr>
</table>


<table>
  <tr>
    <th>Product Name</th>
    <th>Price</th>
  </tr>
  <tr>
    <td>Surface Pro 9</td>
     <td>&#36;999.99</td>
  </tr>
  <tr>
    <td>Surface Laptop 5</td>
    <td>&#36;899.99</td>
  </tr>
  <tr>
    <td>Surface Slim Pen 2</td>
    <td>&#36;94.99</td>
  </tr>
  <tr>
    <td>Surface Laptop SE</td>
    <td>&#36;299.99</td>
  </tr>
  <tr>
    <td>Surface Laptop Go 2</td>
    <td>&#36;599.99</td>
  </tr>
  <tr>
    <td>Surface Go 3</td>
    <td>&#36;384.99</td>
  </tr>
  <tr>
    <td>Surface Studio 2+</td>
    <td>&#36;1199.99</td>
  </tr>
  <tr>
    <td>Surface Pen</td>
    <td>&#36;99.99</td>
  </tr>
  <tr>
    <td>Surface Duo 2</td>
    <td>&#36;1099.99</td>
  </tr>
  <tr>
    <td>Surface Dock</td>
    <td>&#36;199.99</td>
  </tr>
</table>

### Example 2:  Sales - Example of Joins

In [7]:
question_sales = 'How many laptops were sold in December?'
completion_sales = generate_nl_to_sql(question_sales)
print(completion_sales)

SELECT SUM(Sales_Detail.quantity) AS total_laptops_sold
FROM Sales_Detail
JOIN Sales ON Sales_Detail.sales_id = Sales.sale_id
JOIN Products ON Sales_Detail.prod_id = Products.prod_id
WHERE Products.category = 'laptop'
AND MONTH(Sales.date) = 12;


In [8]:
sql_output_sales = run_sql_query(completion_sales)
print(sql_output_sales)

   total_laptops_sold
0              534931


In [9]:
nl_output = generate_sql_to_nl(str(question_sales +  str(sql_output_sales.to_dict('list')) ))
print(nl_output)

In December, a total of 534,931 laptops were sold.


### Example 3: Stock - Example of Multi Turn questions

In [10]:
question_stock = 'How much stock of Surface Laptop 5 are we currently carrying?'
completion_stock = generate_nl_to_sql(question_stock)
print(completion_stock)

SELECT SUM(stock) as total_stock FROM Stock WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%');


In [11]:
sql_output_stock = run_sql_query(completion_stock)
print(sql_output_stock)

   total_stock
0         2527


In [12]:
nl_output = generate_sql_to_nl(str(question_stock +  str(sql_output_stock.to_dict('list')) ))
print(nl_output)

We are currently carrying a total stock of 2,527 units of Surface Laptop 5.


In [13]:
history = []
history.append({"role": "user","content": question_stock})
history.append({"role": "system","content":nl_output})
history

[{'role': 'user',
  'content': 'How much stock of Surface Laptop 5 are we currently carrying?'},
 {'role': 'system',
  'content': 'We are currently carrying a total stock of 2,527 units of Surface Laptop 5.'}]

In [21]:
question_stock_2 = 'How much of this is in GadgetWorld?'
completion_stock_2 = generate_nl_to_sql(question_stock_2, history)
print(completion_stock_2)

SELECT SUM(stock) as total_stock FROM Stock WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%') AND merchant_id IN (SELECT merchant_id FROM Merchants WHERE merchant_name LIKE '%GadgetWorld%');


In [22]:
sql_output_stock_2 = run_sql_query(completion_stock_2)
print(sql_output_stock_2)

   total_stock
0          195


In [27]:
nl_output = generate_sql_to_nl(str(question_stock_2 +  str(sql_output_stock_2.to_dict('list'))))
print(nl_output)

There are 195 items in total stock at GadgetWorld.
